In [4]:
from sklearn import svm, metrics
import random, re

lines=open("iris.csv",'r',encoding='utf-8').read().split('\n')
f_tonum=lambda n: float(n) if re.match(r'^[0-9\.]+$', n) else n 
f_cols=lambda li: list(map(f_tonum,li.strip().split(',')))
csv=list(map(f_cols,lines))
del csv[0] # 헤더제거
random.shuffle(csv)

#데이터 k개로 분할
k=5
csvk=[ [] for i in range(k)]
for i in range(len(csv)):
    csvk[i%k].append(csv[i])

#리스트를 훈련  전용 데이터와 테스트 전용 데이터로 분할
def split_data_label(rows):
    data=[]
    label=[]
    for row in rows:
        data.append(row[0:4])
        label.append(row[4])
    return (data, label)

#정답률
def calc_score(test,train):
    test_f, test_l=split_data_label(test)
    train_f, train_l=split_data_label(train)
    #학습시키고 정답률 구하기
    clf=svm.SVC()
    clf.fit(train_f, train_l)
    pre=clf.predict(test_f)
    return metrics.accuracy_score(test_l,pre)

#k개로 분할해서 정답률 구하기
score_list=[]
for testc in csvk:
    #testc이외의 데이터를 훈련 전용 데이터로 사용
    trainc=[]
    for i in csvk:
        if i != testc: trainc +=i
    sc=calc_score(testc, trainc)
    score_list.append(sc)
    
print("각각의 정답률=", score_list)
print("평균 정답률=", sum(score_list)/ len(score_list))
    

[[[5.5, 2.4, 3.8, 1.1, 'Iris-versicolor'], [6.9, 3.2, 5.7, 2.3, 'Iris-virginica'], [5.0, 3.2, 1.2, 0.2, 'Iris-setosa'], [5.7, 3.0, 4.2, 1.2, 'Iris-versicolor'], [5.6, 2.5, 3.9, 1.1, 'Iris-versicolor'], [6.3, 3.3, 6.0, 2.5, 'Iris-virginica'], [4.5, 2.3, 1.3, 0.3, 'Iris-setosa'], [5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'], [5.7, 2.5, 5.0, 2.0, 'Iris-virginica'], [6.7, 3.0, 5.2, 2.3, 'Iris-virginica'], [6.8, 3.0, 5.5, 2.1, 'Iris-virginica'], [6.3, 2.5, 5.0, 1.9, 'Iris-virginica'], [5.1, 3.4, 1.5, 0.2, 'Iris-setosa'], [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'], [6.1, 3.0, 4.6, 1.4, 'Iris-versicolor'], [6.3, 3.4, 5.6, 2.4, 'Iris-virginica'], [6.3, 2.3, 4.4, 1.3, 'Iris-versicolor'], [6.3, 2.5, 4.9, 1.5, 'Iris-versicolor'], [6.5, 3.0, 5.8, 2.2, 'Iris-virginica'], [6.0, 2.2, 5.0, 1.5, 'Iris-virginica'], [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'], [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'], [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'], [5.9, 3.0, 4.2, 1.5, 'Iris-versicolor'], [4.9, 2.4, 3.3,

In [15]:
#scikitlearn의 crossvalidation
import pandas as pd
from sklearn import svm, metrics, model_selection
import random, re

#iris data 불러오기
csv=pd.read_csv("iris.csv")

#train / test set 분리
data=csv[["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]]
label=csv["Name"]

#cross validation
clf=svm.SVC()
scores=model_selection.cross_val_score(clf, data, label, cv=5)
print("각각의 정답률=", scores)
print("평균 정답률=", scores.mean())


각각의 정답률= [0.96666667 1.         0.96666667 0.96666667 1.        ]
평균 정답률= 0.9800000000000001


In [21]:
# Grid search
import pandas as pd
from sklearn import model_selection, svm, metrics
from sklearn.grid_search import GridSearchCV

#MNIST 데이터 불러오기
train_csv=pd.read_csv("./mnist/train.csv")
test_csv=pd.read_csv("./mnist/t10k.csv")

#필요 열 추출
train_label=train_csv.ix[:,0]
train_data=train_csv.ix[:,1:577]
test_label=test_csv.ix[:,0]
test_data=test_csv.ix[:,1:577]
print("학습 데이터의 수=", len(train_label))

#그리드서치 매개변수 설정 #후보지정
params=[
    {"C":[1,10,100,1000],"kernal":["linear"]},
    {"C":[1,10,100,1000],"kernal":["rbf"],"gamma":[0.001,0.0001]}
]

#그리드서치 수행
clf=GridSearchCV(svm.SVC(),params, n_jobs=-1) #GridSerarchCV(분류기, 파라미터후보들, 병렬프로세수 수) / n_jobs=-1일 경우 자동으로 코어수에 맞게 프로세스 수 설정
clf.fit(train_data, train_label)
print("학습기=",clf.best_estimator_)

#테스트 데이터 확인
pre=clf.predict(test_data)
as_score=metrics.accuracy_score(pre,test_label)
print("정답률=",ac_score)

학습 데이터의 수= 1000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
/anaconda3/lib/python3.6/si

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/anaconda3/lib/python3.6/runpy.py in _run